### Priv Attack Implementation - Walkthrough/Playground
Here you can play with and understand how this project is implemented,
lets start by configuring our experiment:

In [1]:
model = 'sac'
environment_humanoid = 'Humanoid-v2'
environment_halfcheetah = 'HalfCheetah-v2'
dim_humanoid=394
dim_halfcheetah=24
seeds = [34, 23] 
num_timesteps = 2000
threshold = 0.7
trajectory_length = 100
attack_model_size = 100
test_size = 50

Note: DeepRL models require lots of training experience before becoming stable and effective at their task, this amount
varies depending on the environment's dimensionality (experience in timesteps):
* HalfCheetah-v2: 3,000,000+
* Humanoid-v2: 10,000,000+ 

By training "shadow models" we attempt to imitate the behaviour of our target DeepRL model. Each seed generates its
own independent model:

In [2]:
from trainer import train_shadow_model
for seed in seeds:
    train_shadow_model(model=model,environment=environment_halfcheetah,
                       seed=seed,timesteps=num_timesteps)

Logging data to output/Humanoid-v2/69/progress.txt
Saving config:

{
    "ac_kwargs":	{},
    "actor_critic":	"MLPActorCritic",
    "alpha":	0.2,
    "batch_size":	100,
    "env_fn":	"<function output_model.<locals>.<lambda> at 0x104e6c1e0>",
    "epochs":	1,
    "exp_name":	"Humanoid-v2_shadow_69",
    "gamma":	0.99,
    "logger":	{
        "<utils.logx.EpochLogger object at 0x104ea8ba8>":	{
            "epoch_dict":	{},
            "exp_name":	"Humanoid-v2_shadow_69",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"output/Humanoid-v2/69",
            "output_file":	{
                "<_io.TextIOWrapper name='output/Humanoid-v2/69/progress.txt' mode='w' encoding='UTF-8'>":	{
                    "mode":	"w"
                }
            }
        }
    },
    "logger_kwargs":	{
        "exp_name":	"Humanoid-v2_shadow_69",
        "output_dir":	"output/Humanoid-v2/69"
    },
    "lr":	0.001,
    "max_ep_len":	10

/usr/local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/usr/local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/usr/local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


With a set of trained models we can prepare a dataset to train our attack model.

To do this we take an equal amount of "training trajectories" and "test trajectories" - experiences during the models
training and testing, from each seed. From these trajectories we pair a "train" and "test" together at random if they
come from the same model (seed) they are labeled with a 1, otherwise 0. 

i.e: 
if the pair (train_trajectory 23, test_trajectory 45) are both from seed 4 - the pair will be labeled with a 1.

This attempts to give the attack model, which is a classifier, the ability to recognize the distinctive characteristics of 
a model's behaviour.

In [3]:
from workers.attack_classifier import train_attack_model_v2
train_attack_model_v2(environment=environment_halfcheetah, threshold=threshold, trajectory_length=trajectory_length,seeds=seeds,
                   attack_model_size=attack_model_size,test_size=test_size,timesteps=num_timesteps, dimension=dim_halfcheetah)





formatting train, eval, test data and labels
concatenating train and test data
pairing and labeling trajectories
generating pairs
saved test pairs
saved train pairs
saved evaluation pairs
creating xgb matricies
xgb eval created
xgb train created
xgb test created
[21:08:48] 80x78800 matrix with 6304000 entries loaded from tmp_plks/491a0a01-bdcd-41b6-9c56-9a8ff185a99d.buffer
[21:08:48] 20x78800 matrix with 1576000 entries loaded from tmp_plks/3b0de896-a15e-4f30-939d-d4ef60f99280.buffer
training classifier
[0]	eval-logloss:0.69355	eval-error:0.55000	eval-rmse:0.50020	train-logloss:0.69060	train-error:0.25000	train-rmse:0.49872
[1]	eval-logloss:0.69396	eval-error:0.55000	eval-rmse:0.50041	train-logloss:0.68809	train-error:0.25000	train-rmse:0.49747
[2]	eval-logloss:0.69294	eval-error:0.35000	eval-rmse:0.49990	train-logloss:0.68422	train-error:0.22500	train-rmse:0.49552
[3]	eval-logloss:0.69337	eval-error:0.55000	eval-rmse:0.50011	train-logloss:0.68178	train-error:0.25000	train-rmse:0.49428

(0.58, 1.0683267638611398, 0.48)